# Forest Dieback Simulation (Field Codex 3.0)

This notebook implements a simple **reaction–diffusion–interference** model for
forest dieback (bossterfte) in a stylised landscape.

It corresponds to the equations in:
`docs/ecology/Bossterfte.md`

Main ideas:

- A tree biomass field `phi_tree[x,y,t]` evolves over time.
- Water `W[x,y]`, nutrients `N[x,y]` and soil stress `S[x,y]` modulate growth.
- Heat `T[x,y]`, drought `D[x,y]`, pests `P_pest[x,y]` and fragmentation `F_frag[x,y]`
  act as stress fields.
- An interference term models **synergy** between stressors.
- We track:
  - average tree biomass
  - forest coherence C(t)
  - total interference load I(t)


In [0]:
import numpy as np
import matplotlib.pyplot as plt

# For reproducibility
rng = np.random.default_rng(123)

# Grid size (stylised forest landscape)
N = 80  # 80x80 grid

# Time steps
T = 200

# Model parameters
D_tree = 0.1    # diffusion rate for tree biomass (seed flow / local spread)
alpha = 0.4     # intrinsic growth rate
beta_S = 0.6    # soil stress mortality coefficient
beta_T = 0.5    # heat mortality coefficient
beta_D = 0.5    # drought mortality coefficient

# Interference coefficients (synergy between stressors)
gamma_TD = 0.9   # heat × drought
gamma_TS = 0.6   # heat × soil stress
gamma_DP = 0.7   # drought × pests
gamma_PF = 0.6   # pests × fragmentation

# Coherence parameters
xi = 5.0            # coherence length (in grid units)
forest_threshold = 0.3  # minimal biomass to count as 'forest'


## Initialise fields

We define:

- `phi_tree`: tree biomass field
- `W`: water availability (0–1)
- `N`: nutrient availability (0–1)
- `S`: soil stress (0–1)
- `T`: heat stress pattern (0–1)
- `D`: drought stress pattern (0–1)
- `P_pest`: pest pressure (0–1)
- `F_frag`: fragmentation / edge stress (0–1)


In [0]:
x = np.linspace(-1, 1, N)
X, Y = np.meshgrid(x, x)

# Water availability: lower in an elevated centre region, higher at 'valleys'
W = 0.7 - 0.6 * np.exp(-((X**2 + Y**2) / (2 * 0.4**2)))
W += 0.05 * rng.normal(size=(N, N))
W = np.clip(W, 0.0, 1.0)

# Nutrient availability: somewhat aligned with water, but with noise
N_field = 0.5 + 0.3 * W + 0.1 * rng.normal(size=(N, N))
N_field = np.clip(N_field, 0.0, 1.0)

        # Soil stress S: higher near a 'road' or disturbance band
S = np.zeros((N, N))
road_band = np.exp(-((Y + 0.2)**2) / (2 * 0.08**2))
S += 0.8 * road_band
S += 0.1 * rng.normal(size=(N, N))
S = np.clip(S, 0.0, 1.0)

# Heat stress T: stronger in lower-left part of the domain
T_field = 0.3 + 0.4 * np.exp(-((X + 0.5)**2 + (Y + 0.5)**2) / (2 * 0.35**2))
T_field += 0.05 * rng.normal(size=(N, N))
T_field = np.clip(T_field, 0.0, 1.0)

# Drought stress D: correlated with low water
D_field = 1.0 - W
D_field += 0.05 * rng.normal(size=(N, N))
D_field = np.clip(D_field, 0.0, 1.0)

# Pest pressure P_pest: clusters in some parts of the forest
P_pest = np.zeros((N, N))
for (cx, cy, r) in [(-0.4, 0.0, 0.25), (0.3, 0.5, 0.2)]:
    dist2 = (X - cx)**2 + (Y - cy)**2
    P_pest += np.exp(-dist2 / (2 * r**2))
P_pest = P_pest - P_pest.min()
if P_pest.max() > 0:
    P_pest = P_pest / (P_pest.max() + 1e-8)

# Fragmentation field F_frag: higher towards domain edges
edge_distance = np.minimum.reduce([
    X - X.min(), X.max() - X,
    Y - Y.min(), Y.max() - Y
])
edge_distance = (edge_distance - edge_distance.min()) / (edge_distance.max() - edge_distance.min() + 1e-8)
F_frag = 1.0 - edge_distance  # 1 at edges, 0 in centre

# Carrying capacity K depends on water and nutrients
K = 0.5 + 2.0 * W * (0.5 + 0.5 * N_field)

# Initial tree biomass: forest patches where W and N are reasonably high
phi_tree = 0.6 * K * (W > 0.3) * (N_field > 0.3)
phi_tree *= 0.8 + 0.4 * rng.normal(size=(N, N))
phi_tree = np.clip(phi_tree, 0.0, None)


## Utility functions

### 1. Discrete Laplacian (diffusion)
Periodic boundary conditions for simplicity.

### 2. Forest coherence C(t)

We approximate forest coherence as a local weighted fraction of cells
with biomass above a threshold, using an exponential distance kernel.


In [0]:
from scipy.ndimage import convolve

def laplacian(field):
    """Simple 2D Laplacian with periodic boundaries."""
    kernel = np.array([[0, 1, 0],
                       [1, -4, 1],
                       [0, 1, 0]])
    return convolve(field, kernel, mode="wrap")


def forest_coherence(phi, threshold=0.3, xi=5.0, window=7):
    """Approximate spatial coherence of forest biomass field.

    phi: tree biomass field (N x N)
    threshold: minimum biomass to count as forest
    xi: coherence length
    window: local neighbourhood size (odd integer)
    """
    N = phi.shape[0]
    half = window // 2

    # Precompute distance-based weights
    coords = np.arange(-half, half+1)
    dX, dY = np.meshgrid(coords, coords)
    dist = np.sqrt(dX**2 + dY**2)
    weights = np.exp(-dist / max(xi, 1e-6))
    weights /= weights.sum()

    # Binary mask: where forest is present
    mask = (phi > threshold).astype(float)

    # Local weighted forest fraction
    local_forest = convolve(mask, weights, mode="wrap")

    # Average over all cells
    C = local_forest.mean()
    return C


## Simulation loop

We implement a discrete version of the PDE:

\[
\frac{\partial \Phi}{\partial t}
= D_{tree} \nabla^2 \Phi
+ \alpha f_W(W) f_N(N) \Phi (1 - \Phi / K)
- \beta_S S \Phi
- \beta_T h_T(T) \Phi
- \beta_D h_D(D) \Phi
- I_{tot}
\]

with interference:

\[
I_{tot} = (\gamma_{TD} T D
+ \gamma_{TS} T S
+ \gamma_{DP} D P_{pest}
+ \gamma_{PF} P_{pest} F_{frag}) \Phi.
\]


In [0]:
def f_W(W):
    W0 = 0.3
    return W / (W + W0)

def f_N(N_field):
    N0 = 0.3
    return N_field / (N_field + N0)

def h_T(T_field):
    # threshold-like response: negligible below 0.3, sharp above
    return np.clip((T_field - 0.3) / 0.7, 0.0, 1.0)

def h_D(D_field):
    # similar threshold-like response for drought
    return np.clip((D_field - 0.3) / 0.7, 0.0, 1.0)

phi_t = phi_tree.copy()

mean_phi_series = []
coherence_series = []
interference_series = []

for t in range(T):
    # Diffusion
    diff = D_tree * laplacian(phi_t)

    # Growth term
    growth = alpha * f_W(W) * f_N(N_field) * phi_t * (1.0 - phi_t / (K + 1e-8))

    # Mortality terms
    mort_S = beta_S * S * phi_t
    mort_T = beta_T * h_T(T_field) * phi_t
    mort_D = beta_D * h_D(D_field) * phi_t

    # Interference terms (synergy between stressors)
    I_tot = (
        gamma_TD * T_field * D_field * phi_t +
        gamma_TS * T_field * S * phi_t +
        gamma_DP * D_field * P_pest * phi_t +
        gamma_PF * P_pest * F_frag * phi_t
    )

    # Update field (Euler step)
    phi_t = phi_t + diff + growth - mort_S - mort_T - mort_D - I_tot
    phi_t = np.clip(phi_t, 0.0, None)

    # Record metrics
    mean_phi_series.append(phi_t.mean())
    coherence_series.append(forest_coherence(phi_t, threshold=forest_threshold, xi=xi))
    interference_series.append(I_tot.mean())


## Visualisation

- Time series of mean forest biomass
- Interference load
- Forest coherence C(t)
- Final forest field snapshot


In [0]:
fig, axes = plt.subplots(2, 2, figsize=(10, 8))
ax1, ax2, ax3, ax4 = axes.ravel()

# 1. Mean forest biomass over time
ax1.plot(mean_phi_series)
ax1.set_title("Mean forest biomass")
ax1.set_xlabel("Time step")
ax1.set_ylabel("Mean Φ_tree")

# 2. Interference over time
ax2.plot(interference_series)
ax2.set_title("Average interference load")
ax2.set_xlabel("Time step")
ax2.set_ylabel("I_tot (mean)")

# 3. Forest coherence C(t)
ax3.plot(coherence_series)
ax3.set_title("Forest coherence C(t)")
ax3.set_xlabel("Time step")
ax3.set_ylabel("C")

# 4. Final forest field
im = ax4.imshow(phi_t, origin="lower")
ax4.set_title("Final forest field Φ_tree(x,y,T)")
plt.colorbar(im, ax=ax4, fraction=0.046, pad=0.04)

plt.tight_layout()
plt.show()


## Experimenting with scenarios

You can now modify the fields and parameters and re-run the simulation cell:

- Reduce `T_field` or `D_field` to simulate fewer heatwaves/droughts.
- Reduce `P_pest` or change its spatial pattern.
- Reduce `S` (soil stress) to model better soil restoration.
- Change the initial `phi_tree` to represent more fragmented or more coherent forest.
- Change `forest_threshold` in `forest_coherence` to test sensitivity.

You should see:

- High-stress, high-interference regimes → collapse of `mean_phi_series` and dropping coherence.
- Restoration scenarios → stabilisation of biomass and higher coherence.

This notebook expresses the **Field Codex 3.0** idea for forests:

> Forest dieback is not just fewer trees, but a collapse of **field structure, coherence and resilience** under combined stressors.
